<a href="https://colab.research.google.com/github/Kenza1525/AI-Powered-Disaster-Response/blob/main/CrisisVLMNetMultimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install required packages for
!pip install -qq torch==2.6.0 torchvision==0.21.0 transformers==4.48.2 \
accelerate==1.3.0 flash_attn==2.7.4.post1 peft==0.13.2 \
soundfile==0.13.1 pillow==11.1.0 scipy==1.15.2 backoff==2.2.1 \
wandb tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 108.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
  

In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Core imports
import re
import gc
import os
import json
import random
import getpass
import wandb
import shutil
import zipfile
import random
import math
import torch
import time
import logging
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from transformers import ElectraTokenizer
from transformers import ElectraModel, ElectraConfig
from transformers import AutoTokenizer, AutoProcessor, AutoModelForCausalLM, GenerationConfig
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support, classification_report

In [ ]:
torch.backends.cudnn.benchmark = True


In [ ]:
#Set seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

#Check device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

#W&B Login (use getpass to hide API key)
wandb_key = getpass.getpass("Enter your W&B API key: ")
wandb.login(key=wandb_key)


Using device: cuda
Enter your W&B API key: ··········


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lnzabamp (principle-paper) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
#Master configuration dictionary
config = {
    "task": "task1",  # Options: task1, task2, task3
    "setting": "A",   # Options: A or B

    "model_type": "CrisisVLMNet",  # Model variants: CrisisVLMNet, TextOnly, etc.
    "use_caption": True,
    "use_image": True,

    #Training hyperparameters
    "max_epochs": 20,
    "batch_size": 16,
    "learning_rate": 1e-5,
    "weight_decay": 0.05,
    "optimizer": "AdamW",

    #W&B config
    "log_to_wandb": True,
    "wandb_project": "ai-disaster-response",
    "wandb_entity": "principle-paper",

    #Paths
    "root_dir": "/content/drive/MyDrive/AI_Disaster_Management",
    "dataset_dir": "/content/drive/MyDrive/AI_Disaster_Management/CrisisMMD/CrisisMMD_v2.0",
    "caption_path": "/content/drive/MyDrive/AI_Disaster_Management/captions/captions_phi4.json",
    "split_dir": "/content/drive/MyDrive/AI_Disaster_Management/CrisisMMD/CrisisMMD_v2.0/splits/crisismmd_datasplit_all",
    "model_save_path": "/content/drive/MyDrive/AI_Disaster_Management/models",
    "image_dir": "/content/drive/MyDrive/AI_Disaster_Management/CrisisMMD/CrisisMMD_v2.0/data_image",
    "split_dir": "/content/drive/MyDrive/AI_Disaster_Management/CrisisMMD/CrisisMMD_v2.0/splits/crisismmd_datasplit_all",
    "enriched_split_dir": "/content/drive/MyDrive/AI_Disaster_Management/CrisisMMD/CrisisMMD_v2.0/splits/enriched_phi4",
    "device": DEVICE
}

print("Configuration loaded.")


Configuration loaded.


In [ ]:
# #Load Phi-4 Multimodal model
# phi_model_id = "microsoft/Phi-4-multimodal-instruct"

# print("Loading Phi-4 model and processor...")
# phi_processor = AutoProcessor.from_pretrained(phi_model_id, trust_remote_code=True)
# phi_model = AutoModelForCausalLM.from_pretrained(
#     phi_model_id,
#     device_map="cuda" if config["device"] == "cuda" else "auto",
#     torch_dtype="auto",
#     trust_remote_code=True
# ).to(config["device"])

# phi_generation_config = GenerationConfig.from_pretrained(phi_model_id)

# #Structured disaster prompt
# system_prompt = "You are a helpful assistant."
# user_prompt = """Analyze this disaster image and provide a detailed description that includes:
# 1. Main disaster type visible (flood, fire, earthquake damage, hurricane, etc.)
# 2. Severity assessment (severe, moderate, or minor damage)
# 3. Key infrastructure visible and its condition (buildings, roads, bridges, power lines)
# 4. People present and their situation (trapped, injured, being rescued)
# 5. Environmental conditions (water levels, smoke, debris)
# 6. Any visible hazards that emergency responders should be aware of
# 7. Geographic identifiers or landmarks if recognizable
# 8. Temporal indicators (time of day, weather conditions)
# Format the description as a single cohesive paragraph that would help emergency responders understand the situation. Focus on objective, factual observations rather than assumptions."""

# #Generate caption for a single image
# def generate_caption(image_path):
#     from PIL import Image

#     image = Image.open(image_path).convert("RGB")
#     content_list = [
#         {"type": "image", "content": image},
#         {"type": "text", "content": user_prompt, "role": "user"}
#     ]

#     prompt = "<|system|>" + system_prompt + "<|end|>" + "<|user|><|image_1|>" + user_prompt + "<|end|><|assistant|>"

#     inputs = phi_processor(
#         text=prompt,
#         images=[image],
#         return_tensors='pt'
#     ).to(config["device"])

#     with torch.no_grad():
#         output_ids = phi_model.generate(
#             **inputs,
#             max_new_tokens=512,
#             generation_config=phi_generation_config
#         )

#     # Only return new generated tokens
#     output_ids = output_ids[:, inputs["input_ids"].shape[1]:]
#     caption = phi_processor.batch_decode(output_ids, skip_special_tokens=True)[0]
#     return caption

# print("Phi-4 model ready for captioning.")


In [ ]:
# #Pick a random image
# test_image_path = "/content/drive/MyDrive/AI_Disaster_Management/CrisisMMD/CrisisMMD_v2.0/data_image/california_wildfires/10_10_2017/917791044158185473_0.jpg"
# #Confirm full path
# print(f"Using image: {test_image_path}")

# # Display the image
# img = Image.open(test_image_path).convert("RGB")
# plt.imshow(img)
# plt.axis("off")
# plt.title("CrisisMMD Sample Image")
# plt.show()

# #Generate caption using Phi-4
# print("Generating caption with Phi-4...")
# caption = generate_caption(test_image_path)

# #Show caption
# print("\nGenerated Caption:\n")
# print(caption)


In [ ]:
# caption_output_path = config["caption_path"]
# image_root = config["image_dir"]

# #Load existing captions
# if os.path.exists(caption_output_path):
#     with open(caption_output_path, "r") as f:
#         captions = json.load(f)
#     print(f"Loaded {len(captions)} existing captions.")
# else:
#     captions = {}

# #Caption generation loop
# new_captions = 0
# total_images = 0

# for root, _, files in os.walk(image_root):
#     for fname in files:
#         if fname.endswith(".jpg") and not fname.startswith("._"):
#             total_images += 1
#             #Relative path = key for captions dict
#             rel_path = os.path.relpath(os.path.join(root, fname), image_root)

#             if rel_path in captions:
#                 continue  # already done

#             abs_path = os.path.join(root, fname)
#             try:
#                 caption = generate_caption(abs_path)
#                 captions[rel_path] = caption
#                 new_captions += 1
#             except Exception as e:
#                 print(f"Error on {rel_path}: {e}")

#             if new_captions % 100 == 0:
#                 with open(caption_output_path, "w") as f:
#                     json.dump(captions, f, indent=2)

# #Final save
# with open(caption_output_path, "w") as f:
#     json.dump(captions, f, indent=2)

# print(f"\n Finished captioning.")
# print(f"Total images scanned: {total_images}")
# print(f"New captions generated: {new_captions}")
# print(f"Captions saved to: {caption_output_path}")


In [ ]:

# # === Paths ===
# original_split_dir = config["split_dir"]
# enriched_split_dir = "/content/drive/MyDrive/AI_Disaster_Management/CrisisMMD/CrisisMMD_v2.0/splits/enriched_phi4"
# caption_path = config["caption_path"]

# # Make output folder if it doesn't exist
# os.makedirs(enriched_split_dir, exist_ok=True)

# # === Load captions ===
# with open(caption_path, "r") as f:
#     captions = json.load(f)

# print(f"Loaded {len(captions)} captions from Phi-4")

# # === Tweet text cleaning (same as preprocess.py) ===
# def clean_text(text):
#     text = re.sub(r'^RT[\s]+', '', text)
#     text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
#     text = re.sub(r'#', '', text)
#     text = re.sub(r'@\w+', '', text)  # <-- NEW: remove @mentions
#     return text.strip()

# # === Process each original .tsv ===
# tsv_files = [f for f in os.listdir(original_split_dir) if f.endswith(".tsv")]

# for fname in tqdm(tsv_files, desc="Processing TSV splits"):
#     input_path = os.path.join(original_split_dir, fname)
#     df = pd.read_csv(input_path, sep="\t")

#     # Dynamic required column check
#     if "task_damage" in fname:
#         required_cols = ['event_name', 'tweet_id', 'image_id', 'tweet_text', 'image', 'label']
#     else:
#         required_cols = ['event_name', 'tweet_id', 'image_id', 'tweet_text', 'image', 'label',
#                          'label_text', 'label_image', 'label_text_image']

#     if not all(col in df.columns for col in required_cols):
#         print(f"Skipping {fname} — missing required columns.")
#         continue

#     # Clean tweet and add caption if available
#     final_texts = []
#     keep_indices = []

#     for i, row in df.iterrows():
#         cleaned = clean_text(str(row['tweet_text']))
#         rel_path = row['image'].replace("data_image/", "")

#         caption = captions.get(rel_path)
#         if caption:
#             final_texts.append(f"{cleaned} [SEP] {caption}")
#             keep_indices.append(i)

#     # Keep only rows with captions
#     df_filtered = df.iloc[keep_indices].copy()
#     df_filtered["tweet_text"] = df_filtered["tweet_text"].apply(clean_text)
#     df_filtered["final_text"] = final_texts

#     # Save
#     out_path = os.path.join(enriched_split_dir, fname)
#     df_filtered.to_csv(out_path, sep="\t", index=False)
#     print(f"Saved enriched TSV: {out_path} ({len(df_filtered)} rows)")


In [ ]:
# import pandas as pd

# df = pd.read_csv("/content/drive/MyDrive/AI_Disaster_Management/CrisisMMD/CrisisMMD_v2.0/splits/crisismmd_datasplit_all/task_damage_text_img_dev.tsv", sep="\t")
# print(df.columns)
# print(df["image"].head(3))

In [ ]:
# import json

# with open(config["caption_path"]) as f:
#     captions = json.load(f)

# sample_keys = list(captions.keys())[:5]
# print(sample_keys)


In [ ]:
# Clean tweet text (previously in preprocess.py)
def clean_text(text):
    text = re.sub(r'^RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'@\w+', '', text)
    return text.strip()

# Expand to square (from base_dataset.py)
def expand2square(pil_img, background_color=(0, 0, 0)):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

# Label mappings
LABELS = {
    'task1': {
        'informative': 1,
        'not_informative': 0
    },
    'task2': {
        'infrastructure_and_utility_damage': 0,
        'not_humanitarian': 1,
        'other_relevant_information': 2,
        'rescue_volunteering_or_donation_effort': 3,
        'vehicle_damage': 4,
        'affected_individuals': 5,
        'injured_or_dead_people': 5,
        'missing_or_found_people': 5
    },
    'task2_full': {
        'infrastructure_and_utility_damage': 0,
        'not_humanitarian': 1,
        'other_relevant_information': 2,
        'rescue_volunteering_or_donation_effort': 3,
        'vehicle_damage': 4,
        'affected_individuals': 5,
        'injured_or_dead_people': 6,
        'missing_or_found_people': 7
    },
    'task3': {
        'little_or_no_damage': 0,
        'mild_damage': 1,
        'severe_damage': 2
    }
}

# Task name to file prefix mapping
TASK_MAP = {
    'task1': 'informative',
    'task2': 'humanitarian',
    'task2_full': 'humanitarian',
    'task3': 'damage'
}

class CrisisMMDataset(Dataset):
    def __init__(self):
        self.data = []

    def initialize(self, opt, phase='train', cat='all', task='task1'):
        self.task = task
        self.label_map = LABELS[task]
        self.tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')
        self.max_len = opt.get('max_len', 512)
        self.debug = opt.get('debug', False)

        # Fix filename using mapping
        task_file_prefix = TASK_MAP[task]
        tsv_path = os.path.join(opt['enriched_split_dir'], f"task_{task_file_prefix}_text_img_{phase}.tsv")
        df = pd.read_csv(tsv_path, sep='\t')

        if self.debug:
            df = df.head(100)

        self.image_root = opt['image_dir']

        # Read and store
        self.data = []
        for _, row in df.iterrows():
            text = clean_text(row['final_text'])
            image_rel_path = row['image']
            if image_rel_path.startswith("data_image/"):
              image_rel_path = image_rel_path[len("data_image/"):]
            image_path = os.path.join(self.image_root, image_rel_path)

            # Tokenize
            tokens = self.tokenizer(
                text,
                padding='max_length',
                truncation=True,
                max_length=self.max_len,
                return_tensors='pt'
            )

            self.data.append({
                'input_ids': tokens['input_ids'].squeeze(0),
                'attention_mask': tokens['attention_mask'].squeeze(0),
                'token_type_ids': tokens['token_type_ids'].squeeze(0),
                'image_path': image_path,
                'label': torch.tensor(self.label_map[row['label']], dtype=torch.long),
                'text': text  # optional
            })

        # Define transforms
        self.transforms = transforms.Compose([
            transforms.Lambda(lambda img: expand2square(img)),
            transforms.Resize((opt.get('load_size', 224), opt.get('load_size', 224))),
            transforms.RandomHorizontalFlip(0.2),
            transforms.RandomCrop((opt.get('crop_size', 224), opt.get('crop_size', 224))),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

        print(f"Loaded {len(self.data)} samples for {phase}")

    def __getitem__(self, idx):
        sample = self.data[idx]

        if not os.path.exists(sample['image_path']):
          print(f"[Missing Image] {sample['image_path']}")

        with Image.open(sample['image_path']).convert('RGB') as img:
            image_tensor = self.transforms(img)

        return {
            'input_ids': sample['input_ids'],
            'attention_mask': sample['attention_mask'],
            'token_type_ids': sample['token_type_ids'],
            'image': image_tensor,
            'label': sample['label'],
            'text': sample['text']
        }

    def __len__(self):
        return len(self.data)

    def name(self):
        return "CrisisMMDataset"


In [ ]:
# Base class to handle saving/loading
class BaseModel(nn.Module):
    def __init__(self, save_dir):
        super(BaseModel, self).__init__()
        self.save_dir = save_dir

    def save(self, filename):
        path = os.path.join(self.save_dir, f"{filename}.pt")
        torch.save(self.state_dict(), path)
        print(f" Model saved to: {path}")

    def load(self, filepath):
        self.load_state_dict(torch.load(filepath, map_location='cpu'), strict=False)
        print(f"Loaded model from: {filepath}")

# Text-only ELECTRA model
class TextOnlyModel(BaseModel):
    def __init__(self, save_dir, dim_text_repr=768, num_class=2):
        super(TextOnlyModel, self).__init__(save_dir)

        self.textEncoder = ElectraModel.from_pretrained('google/electra-base-discriminator')


        self.dropout = nn.Dropout()
        self.linear = nn.Linear(dim_text_repr, num_class)

    def forward(self, x):
        _, text = x
        hidden_states = self.textEncoder(**text)
        cls_token = self.dropout(hidden_states[0][:, 0, :])
        return self.linear(cls_token)

# Image-only DenseNet model
class ImageOnlyModel(BaseModel):
    def __init__(self, save_dir, dim_visual_repr=1000, num_class=2):
        super(ImageOnlyModel, self).__init__(save_dir)

        self.imageEncoder = torch.hub.load('pytorch/vision:v0.8.0', 'densenet201', pretrained=True)
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout()
        self.linear = nn.Linear(dim_visual_repr, num_class)

    def forward(self, x):
        image, _ = x
        visual_feats = self.dropout(self.flatten(self.imageEncoder(image)))
        return self.linear(visual_feats)

class MultiTaskDenseNetElectraModel(BaseModel):
    def __init__(self, save_dir, dim_visual_repr=1000, dim_text_repr=768, dim_proj=100,
                 num_classes_task1=2, num_classes_task2=6, num_classes_task3=3):
        super(MultiTaskDenseNetElectraModel, self).__init__(save_dir)

        self.imageEncoder = torch.hub.load('pytorch/vision:v0.8.0', 'densenet201', pretrained=True)
        self.textEncoder = ElectraModel.from_pretrained('google/electra-base-discriminator')

        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(p=0.6)

        # Shared projection layers
        self.proj_visual = nn.Linear(dim_visual_repr, dim_proj)
        self.proj_text = nn.Linear(dim_text_repr, dim_proj)
        self.bn_visual = nn.BatchNorm1d(dim_proj)
        self.bn_text = nn.BatchNorm1d(dim_proj)

        # Attention layers
        self.attn_visual = nn.Linear(dim_visual_repr, dim_proj)
        self.attn_text = nn.Linear(dim_text_repr, dim_proj)

        # Shared feature fusion
        self.fc_attn = nn.Linear(2 * dim_proj, 2 * dim_proj)
        self.bn_attn = nn.BatchNorm1d(2 * dim_proj)

        # Task-specific classification heads
        self.classifier_task1 = nn.Linear(2 * dim_proj, num_classes_task1)
        self.classifier_task2 = nn.Linear(2 * dim_proj, num_classes_task2)
        self.classifier_task3 = nn.Linear(2 * dim_proj, num_classes_task3)

    def apply_self_attention(self, x):
        scores = torch.matmul(x, x.transpose(-1, -2))
        weights = F.softmax(scores, dim=-1)
        return torch.matmul(weights, x)

    def forward(self, x, task=None):
        image, text = x

        visual_feats = self.dropout(self.flatten(self.imageEncoder(image)))
        text_hidden = self.textEncoder(**text)[0][:, 0, :]
        text_hidden = self.dropout(text_hidden)

        # Self-attention and feature extraction (same for all tasks)
        v_attn = self.apply_self_attention(visual_feats)
        t_attn = self.apply_self_attention(text_hidden)

        v_proj = F.relu(self.bn_visual(self.proj_visual(v_attn)))
        t_proj = F.relu(self.bn_text(self.proj_text(t_attn)))

        alpha_v = torch.sigmoid(self.attn_text(t_attn))
        alpha_t = torch.sigmoid(self.attn_visual(v_attn))

        masked_v = alpha_v * v_proj
        masked_t = alpha_t * t_proj

        joint = torch.cat((masked_v, masked_t), dim=1)
        joint = self.dropout(F.relu(self.bn_attn(self.fc_attn(joint))))

        # Return predictions for specific task or all tasks
        if task == "task1":
            return self.classifier_task1(joint)
        elif task == "task2":
            return self.classifier_task2(joint)
        elif task == "task3":
            return self.classifier_task3(joint)
        else:
            # Return all task predictions
            return {
                "task1": self.classifier_task1(joint),
                "task2": self.classifier_task2(joint),
                "task3": self.classifier_task3(joint)
            }

In [ ]:
# Prepare datasets for all tasks
train_set_task1 = CrisisMMDataset()
train_set_task1.initialize(config, phase='train', task="task1")
train_set_task2 = CrisisMMDataset()
train_set_task2.initialize(config, phase='train', task="task2")
train_set_task3 = CrisisMMDataset()
train_set_task3.initialize(config, phase='train', task="task3")

dev_set_task1 = CrisisMMDataset()
dev_set_task1.initialize(config, phase='dev', task="task1")
dev_set_task2 = CrisisMMDataset()
dev_set_task2.initialize(config, phase='dev', task="task2")
dev_set_task3 = CrisisMMDataset()
dev_set_task3.initialize(config, phase='dev', task="task3")

test_set_task1 = CrisisMMDataset()
test_set_task1.initialize(config, phase='test', task="task1")
test_set_task2 = CrisisMMDataset()
test_set_task2.initialize(config, phase='test', task="task2")
test_set_task3 = CrisisMMDataset()
test_set_task3.initialize(config, phase='test', task="task3")


# Create dataloaders
train_loader_task1 = DataLoader(train_set_task1, batch_size=config["batch_size"], shuffle=True)
train_loader_task2 = DataLoader(train_set_task2, batch_size=config["batch_size"], shuffle=True)
train_loader_task3 = DataLoader(train_set_task3, batch_size=config["batch_size"], shuffle=True)

dev_loader_task1 = DataLoader(dev_set_task1, batch_size=config["batch_size"], shuffle=False)
dev_loader_task2 = DataLoader(dev_set_task2, batch_size=config["batch_size"], shuffle=False)
dev_loader_task3 = DataLoader(dev_set_task3, batch_size=config["batch_size"], shuffle=False)

test_loader_task1 = DataLoader(test_set_task1, batch_size=config["batch_size"], shuffle=False)
test_loader_task2 = DataLoader(test_set_task2, batch_size=config["batch_size"], shuffle=False)
test_loader_task3 = DataLoader(test_set_task3, batch_size=config["batch_size"], shuffle=False)




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loaded 8533 samples for train
Loaded 8562 samples for train
Loaded 1705 samples for train
Loaded 1433 samples for dev
Loaded 1436 samples for dev
Loaded 383 samples for dev
Loaded 1434 samples for test
Loaded 1402 samples for test
Loaded 372 samples for test


In [ ]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

def create_optimizer_and_scheduler(config, model, total_steps=None):
    optimizer = AdamW(
        model.parameters(),
        lr=config["learning_rate"],
        weight_decay=config["weight_decay"]
    )

    # Scheduler — Reduce LR on plateau of val_loss
    scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.7,     # More gradual reduction (0.2 instead of 0.1)
    patience=3,     # Wait fewer epochs before reducing (3 instead of 5)
    cooldown=1,     # Shorter cooldown (1 instead of 2)
    min_lr=1e-7,    # Add a minimum learning rate
    verbose=True
)

    return optimizer, scheduler

In [ ]:
# ---- Checkpointing Utilities ----
def save_model(model, optimizer, scheduler, metric, epoch, path='./checkpoint.pth'):
    torch.save(
        {'model_state_dict'      : model.state_dict(),
         'optimizer_state_dict'  : optimizer.state_dict(),
         'scheduler_state_dict'  : scheduler.state_dict(),
         metric[0]               : metric[1],
         'epoch'                 : epoch},
         path
    )

def load_model(path, model, metric= 'val_loss', optimizer= None, scheduler= None):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler is not None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch = checkpoint['epoch']
    metric_value = checkpoint[metric]

    return model, optimizer, scheduler, epoch, metric_value


In [ ]:
from torch.nn.utils import clip_grad_norm_
class MultiTaskTrainer:
    def __init__(self, train_loaders, dev_loaders, test_loaders, model, loss_fn, optimizer, scheduler, config):
        # train_loaders, dev_loaders, test_loaders are dictionaries mapping task names to dataloaders
        self.train_loaders = train_loaders
        self.dev_loaders = dev_loaders
        self.test_loaders = test_loaders
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = config["device"]
        self.save_dir = config["model_save_path"]
        self.config = config

        # Add task weights to config before creating the trainer
        config["task_weights"] = {"task1": 1.0, "task2": 1.0, "task3": 1.0}

        # Or with dataset size weighting
        num_samples = {
            "task1": len(train_set_task1),
            "task2": len(train_set_task2),
            "task3": len(train_set_task3)
        }
        total_samples = sum(num_samples.values())

        self.task_weights = {
            task: (total_samples / (3 * count))
            for task, count in num_samples.items()
}

        # WandB setup
        if config.get("log_to_wandb", False):
            self.run = wandb.init(
                project=config["wandb_project"],
                entity=config["wandb_entity"],
                name=f"multitask_{int(time.time())}",
                config=config,
                reinit=True
            )
        else:
            self.run = None

        if self.device != 'cpu':
            self.scaler = torch.cuda.amp.GradScaler()

    def train(self, max_epochs):
        best_dev_loss = float('inf')
        patience = 7
        patience_counter = 0

        for epoch in range(max_epochs):
            self.model.train()
            total_loss = 0
            task_losses = {task: 0 for task in self.train_loaders}
            task_corrects = {task: 0 for task in self.train_loaders}
            task_totals = {task: 0 for task in self.train_loaders}

            # Determine the length of the longest loader to set as epoch length
            max_batches = max([len(loader) for loader in self.train_loaders.values()])
            loaders_iter = {task: iter(loader) for task, loader in self.train_loaders.items()}

            for _ in tqdm(range(max_batches), desc=f"Epoch {epoch}"):
                self.optimizer.zero_grad()
                epoch_loss = 0

                # Process a batch from each task
                for task, iterator in loaders_iter.items():
                    try:
                        batch = next(iterator)
                    except StopIteration:
                        # Reinitialize iterator if exhausted
                        loaders_iter[task] = iter(self.train_loaders[task])
                        batch = next(loaders_iter[task])

                    x = (
                        batch['image'].to(self.device),
                        {
                            "input_ids": batch["input_ids"].to(self.device),
                            "attention_mask": batch["attention_mask"].to(self.device),
                            "token_type_ids": batch["token_type_ids"].to(self.device)
                        }
                    )
                    y = batch["label"].to(self.device)

                    if self.device != 'cpu':
                        with torch.amp.autocast(device_type='cuda'):
                            logits = self.model(x, task=task)
                            loss = self.loss_fn(logits, y) * self.task_weights[task]
                            epoch_loss += loss

                        # Track metrics
                        task_losses[task] += loss.item()
                        preds = torch.argmax(logits, dim=1)
                        task_corrects[task] += (preds == y).sum().item()
                        task_totals[task] += y.size(0)
                    else:
                        logits = self.model(x, task=task)
                        loss = self.loss_fn(logits, y) * self.task_weights[task]
                        epoch_loss += loss

                        # Track metrics
                        task_losses[task] += loss.item()
                        preds = torch.argmax(logits, dim=1)
                        task_corrects[task] += (preds == y).sum().item()
                        task_totals[task] += y.size(0)

                # Backward pass and optimization
                if self.device != 'cpu':
                    self.scaler.scale(epoch_loss).backward()
                    clip_grad_norm_(self.model.parameters(), max_norm=1.0)  # Clip gradients
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                else:
                    epoch_loss.backward()
                    clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                    self.optimizer.step()

                total_loss += epoch_loss.item()

            # Calculate metrics
            avg_loss = total_loss / (max_batches * len(self.train_loaders))
            task_accs = {task: task_corrects[task] / task_totals[task] if task_totals[task] > 0 else 0
                         for task in task_corrects}

            # Log metrics
            logging.info(f"[Epoch {epoch}] Avg Loss: {avg_loss:.4f}")
            for task, acc in task_accs.items():
                logging.info(f"  - {task} Acc: {acc:.4f}")

            if self.run:
                wandb.log({
                    "train_loss": avg_loss,
                    **{f"{task}_acc": acc for task, acc in task_accs.items()},
                    "epoch": epoch
                })

            # Validation
            dev_loss = self.validate(epoch)

            # Save checkpoint
            if dev_loss < best_dev_loss:
                best_dev_loss = dev_loss
                patience_counter = 0
                model_to_save = self.model.module if hasattr(self.model, "module") else self.model

                save_model(
                    model=model_to_save,
                    optimizer=self.optimizer,
                    scheduler=self.scheduler,
                    metric=('val_loss', dev_loss),
                    epoch=epoch,
                    path=os.path.join(self.save_dir, "checkpoint_best.pth")
                )

            else:
              patience_counter += 1
              if patience_counter >= patience:
                  print(f"Early stopping triggered after {epoch+1} epochs")
                  break

            self.scheduler.step(dev_loss)

    def validate(self, epoch=0):
        self.model.eval()
        total_loss = 0
        task_losses = {task: 0 for task in self.dev_loaders}
        task_corrects = {task: 0 for task in self.dev_loaders}
        task_totals = {task: 0 for task in self.dev_loaders}

        with torch.no_grad():
            for task, loader in self.dev_loaders.items():
                for batch in loader:
                    x = (
                        batch['image'].to(self.device),
                        {
                            "input_ids": batch["input_ids"].to(self.device),
                            "attention_mask": batch["attention_mask"].to(self.device),
                            "token_type_ids": batch["token_type_ids"].to(self.device)
                        }
                    )
                    y = batch["label"].to(self.device)

                    logits = self.model(x, task=task)
                    loss = self.loss_fn(logits, y)

                    task_losses[task] += loss.item() * y.size(0)
                    preds = torch.argmax(logits, dim=1)
                    task_corrects[task] += (preds == y).sum().item()
                    task_totals[task] += y.size(0)

                    total_loss += loss.item() * y.size(0)

        # Calculate metrics
        total_examples = sum(task_totals.values())
        avg_loss = total_loss / total_examples if total_examples > 0 else 0
        task_accs = {task: task_corrects[task] / task_totals[task] if task_totals[task] > 0 else 0
                     for task in task_corrects}

        # Log metrics
        logging.info(f"[Val Epoch {epoch}] Avg Loss: {avg_loss:.4f}")
        for task, acc in task_accs.items():
            logging.info(f"  - {task} Acc: {acc:.4f}")

        if self.run:
            wandb.log({
                "val_loss": avg_loss,
                **{f"val_{task}_acc": acc for task, acc in task_accs.items()},
                "epoch": epoch
            })

        return avg_loss

    def predict(self):
        self.model.eval()
        task_predictions = {task: [] for task in self.test_loaders}

        with torch.no_grad():
            for task, loader in self.test_loaders.items():
                for batch in loader:
                    x = (
                        batch['image'].to(self.device),
                        {
                            "input_ids": batch["input_ids"].to(self.device),
                            "attention_mask": batch["attention_mask"].to(self.device),
                            "token_type_ids": batch["token_type_ids"].to(self.device)
                        }
                    )
                    logits = self.model(x, task=task)
                    preds = torch.argmax(logits, dim=1)
                    task_predictions[task].extend(preds.cpu().numpy())

        return task_predictions

In [ ]:
task_output_sizes = {
    "task1": 2,
    "task2": 8,
    "task2_full": 8,
    "task3": 3
}
# Create model
model = MultiTaskDenseNetElectraModel(
    save_dir=config["model_save_path"],
    num_classes_task1=task_output_sizes["task1"],
    num_classes_task2=task_output_sizes["task2"],
    num_classes_task3=task_output_sizes["task3"]
)
model = nn.DataParallel(model).to(config["device"])

# Create dataloaders dict for all tasks
train_loaders = {
    "task1": train_loader_task1,
    "task2": train_loader_task2,
    "task3": train_loader_task3
}
dev_loaders = {
    "task1": dev_loader_task1,
    "task2": dev_loader_task2,
    "task3": dev_loader_task3
}
test_loaders = {
    "task1": test_loader_task1,
    "task2": test_loader_task2,
    "task3": test_loader_task3
}

# Configure optimizer & scheduler
optimizer, scheduler = create_optimizer_and_scheduler(config, model)
loss_fn = nn.CrossEntropyLoss()

# Set task weights (optional - can help balance tasks)
config["task_weights"] = {"task1": 1.0, "task2": 1.0, "task3": 1.0}

# Create multi-task trainer
trainer = MultiTaskTrainer(
    train_loaders=train_loaders,
    dev_loaders=dev_loaders,
    test_loaders=test_loaders,
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    scheduler=scheduler,
    config=config
)

# Train model
trainer.train(config["max_epochs"])

# Get predictions for all tasks
predictions = trainer.predict()
preds_task1 = predictions["task1"]
preds_task2 = predictions["task2"]
preds_task3 = predictions["task3"]

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.8.0
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


<ipython-input-10-1aad7eb4c9f1>:45: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
Epoch 19: 100%|██████████| 536/536 [11:36<00:00,  1.30s/it]


In [ ]:
# task_output_sizes = {
#     "task1": 2,
#     "task2": 6,
#     "task2_full": 8,
#     "task3": 3
# }
# model = MultiTaskDenseNetElectraModel(
#     save_dir=config["model_save_path"],
#     num_class=task_output_sizes[config["task"]]
# )

# # Load the best checkpoint
# checkpoint_path = os.path.join(config["model_save_path"], "best.pt")
# model.load(checkpoint_path)


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.8.0
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loaded model from: /content/drive/MyDrive/AI_Disaster_Management/models/best.pt


In [ ]:
# def count_parameters(model):
#     total = sum(p.numel() for p in model.parameters())
#     trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
#     return total, trainable

# total_params, trainable_params = count_parameters(model)
# print(f"Total parameters: {total_params:,}")
# print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
# After training is finished
print("Loading best model for testing...")
best_model_path = os.path.join(config["model_save_path"], "checkpoint_best.pth")
best_model = MultiTaskDenseNetElectraModel(
    save_dir=config["model_save_path"],
    num_classes_task1=task_output_sizes["task1"],
    num_classes_task2=task_output_sizes["task2"],
    num_classes_task3=task_output_sizes["task3"]
)
best_model = nn.DataParallel(best_model).to(config["device"])
load_model(best_model_path, best_model)

# Create a new test predict function that uses the best model
def predict_with_best_model():
    best_model.eval()
    task_predictions = {task: [] for task in test_loaders}

    with torch.no_grad():
        for task, loader in test_loaders.items():
            for batch in loader:
                x = (
                    batch['image'].to(config["device"]),
                    {
                        "input_ids": batch["input_ids"].to(config["device"]),
                        "attention_mask": batch["attention_mask"].to(config["device"]),
                        "token_type_ids": batch["token_type_ids"].to(config["device"])
                    }
                )
                logits = best_model(x, task=task)
                preds = torch.argmax(logits, dim=1)
                task_predictions[task].extend(preds.cpu().numpy())

    return task_predictions

# Get predictions using the BEST model
print("\nPredicting on test sets using best model...")
predictions = predict_with_best_model()
preds_task1 = predictions["task1"]
preds_task2 = predictions["task2"]
preds_task3 = predictions["task3"]

# extract ground truth and compute metrics
y_true_task1 = [sample["label"].item() for sample in test_set_task1]
y_true_task2 = [sample["label"].item() for sample in test_set_task2]
y_true_task3 = [sample["label"].item() for sample in test_set_task3]

# Compute metrics for each task
metrics_task1 = compute_metrics(y_true_task1, preds_task1, task_name="Task 1 (Informative)", verbose=True)
metrics_task2 = compute_metrics(y_true_task2, preds_task2, task_name="Task 2 (Humanitarian)", verbose=True)
metrics_task3 = compute_metrics(y_true_task3, preds_task3, task_name="Task 3 (Damage)", verbose=True)


# Prepare accuracy values for MTMS calculation
task_accuracies = {
    "task1": metrics_task1["accuracy"],
    "task2": metrics_task2["accuracy"],
    "task3": metrics_task3["accuracy"]
}

# Define number of classes for each task
task_class_counts = {
    "task1": 2,  # informative vs. not_informative
    "task2": 6,  # humanitarian categories
    "task3": 3   # damage severity levels
}

def compute_mtms(task_metrics, task_class_counts):
    """
    Compute Multi-task Model Strength (MTMS) as defined in the CrisisKAN paper.

    Args:
        task_metrics: Dictionary mapping task names to their accuracy values
        task_class_counts: Dictionary mapping task names to their number of classes

    Returns:
        MTMS score (float between 0 and 1)
    """
    total_classes = sum(task_class_counts.values())
    mtms = 0.0

    for task, acc in task_metrics.items():
        beta = task_class_counts[task] / total_classes
        mtms += beta * acc

    print(f"\nMulti-task Model Strength (MTMS): {mtms:.4f}")
    return mtms

# Compute MTMS
mtms = compute_mtms(task_accuracies, task_class_counts)

# Log to WandB if enabled
if trainer.run:
    wandb.log({
        "MTMS": mtms,
        "task1_accuracy": metrics_task1["accuracy"],
        "task1_macro_f1": metrics_task1["macro_f1"],
        "task1_weighted_f1": metrics_task1["weighted_f1"],
        "task2_accuracy": metrics_task2["accuracy"],
        "task2_macro_f1": metrics_task2["macro_f1"],
        "task2_weighted_f1": metrics_task2["weighted_f1"],
        "task3_accuracy": metrics_task3["accuracy"],
        "task3_macro_f1": metrics_task3["macro_f1"],
        "task3_weighted_f1": metrics_task3["weighted_f1"]
    })
    trainer.run.finish()


Loading best model for testing...


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.8.0


RuntimeError: Error(s) in loading state_dict for DataParallel:
	Missing key(s) in state_dict: "module.imageEncoder.features.conv0.weight", "module.imageEncoder.features.norm0.weight", "module.imageEncoder.features.norm0.bias", "module.imageEncoder.features.norm0.running_mean", "module.imageEncoder.features.norm0.running_var", "module.imageEncoder.features.denseblock1.denselayer1.norm1.weight", "module.imageEncoder.features.denseblock1.denselayer1.norm1.bias", "module.imageEncoder.features.denseblock1.denselayer1.norm1.running_mean", "module.imageEncoder.features.denseblock1.denselayer1.norm1.running_var", "module.imageEncoder.features.denseblock1.denselayer1.conv1.weight", "module.imageEncoder.features.denseblock1.denselayer1.norm2.weight", "module.imageEncoder.features.denseblock1.denselayer1.norm2.bias", "module.imageEncoder.features.denseblock1.denselayer1.norm2.running_mean", "module.imageEncoder.features.denseblock1.denselayer1.norm2.running_var", "module.imageEncoder.features.den

In [ ]:
# def compute_metrics(y_true, y_pred, task_name=None, verbose=True):
#     acc = accuracy_score(y_true, y_pred)
#     macro_f1 = f1_score(y_true, y_pred, average='macro')
#     weighted_f1 = f1_score(y_true, y_pred, average='weighted')
#     report = classification_report(y_true, y_pred)

#     if verbose:
#         print(f"\nEvaluation Report for {task_name or 'Unnamed Task'}")
#         print("---------------------------------------")
#         print(f"Accuracy     : {acc:.4f}")
#         print(f"Macro F1     : {macro_f1:.4f}")
#         print(f"Weighted F1  : {weighted_f1:.4f}")
#         print(report)

#     return {
#         "accuracy": acc,
#         "macro_f1": macro_f1,
#         "weighted_f1": weighted_f1,
#         "report": report
#     }

# def compute_mtms(task_metrics, task_class_counts):
#     """
#     Compute Multi-task Model Strength (MTMS) as defined in the CrisisKAN paper.

#     Args:
#         task_metrics: Dictionary mapping task names to their accuracy values
#         task_class_counts: Dictionary mapping task names to their number of classes

#     Returns:
#         MTMS score (float between 0 and 1)
#     """
#     total_classes = sum(task_class_counts.values())
#     mtms = 0.0

#     for task, acc in task_metrics.items():
#         beta = task_class_counts[task] / total_classes
#         mtms += beta * acc

#     print(f"\nMulti-task Model Strength (MTMS): {mtms:.4f}")
#     return mtms


In [ ]:
# # Extract ground truth labels for each task
# y_true_task1 = [sample["label"].item() for sample in test_set_task1]
# y_true_task2 = [sample["label"].item() for sample in test_set_task2]
# y_true_task3 = [sample["label"].item() for sample in test_set_task3]

# # Compute metrics for each task
# metrics_task1 = compute_metrics(y_true_task1, preds_task1, task_name="Task 1 (Informative)", verbose=True)
# metrics_task2 = compute_metrics(y_true_task2, preds_task2, task_name="Task 2 (Humanitarian)", verbose=True)
# metrics_task3 = compute_metrics(y_true_task3, preds_task3, task_name="Task 3 (Damage)", verbose=True)

# # Prepare accuracy values for MTMS calculation
# task_accuracies = {
#     "task1": metrics_task1["accuracy"],
#     "task2": metrics_task2["accuracy"],
#     "task3": metrics_task3["accuracy"]
# }

# # Define number of classes for each task
# task_class_counts = {
#     "task1": 2,  # informative vs. not_informative
#     "task2": 6,  # humanitarian categories
#     "task3": 3   # damage severity levels
# }

# # Compute MTMS
# mtms = compute_mtms(task_accuracies, task_class_counts)

# # Log to WandB if enabled
# if trainer.run:
#     wandb.log({
#         "MTMS": mtms,
#         "task1_accuracy": metrics_task1["accuracy"],
#         "task1_macro_f1": metrics_task1["macro_f1"],
#         "task1_weighted_f1": metrics_task1["weighted_f1"],
#         "task2_accuracy": metrics_task2["accuracy"],
#         "task2_macro_f1": metrics_task2["macro_f1"],
#         "task2_weighted_f1": metrics_task2["weighted_f1"],
#         "task3_accuracy": metrics_task3["accuracy"],
#         "task3_macro_f1": metrics_task3["macro_f1"],
#         "task3_weighted_f1": metrics_task3["weighted_f1"]
#     })
#     trainer.run.finish()


# print("\nTraining and evaluation complete!")


Evaluation Report for Task 1 (Informative)
---------------------------------------
Accuracy     : 0.7943
Macro F1     : 0.7787
Weighted F1  : 0.7940
              precision    recall  f1-score   support

           0       0.72      0.72      0.72       530
           1       0.83      0.84      0.84       904

    accuracy                           0.79      1434
   macro avg       0.78      0.78      0.78      1434
weighted avg       0.79      0.79      0.79      1434


Evaluation Report for Task 2 (Humanitarian)
---------------------------------------
Accuracy     : 0.6341
Macro F1     : 0.4186
Weighted F1  : 0.6178
              precision    recall  f1-score   support

           0       0.49      0.61      0.54       223
           1       0.78      0.78      0.78       536
           2       0.63      0.59      0.61       344
           3       0.50      0.63      0.56       208
           4       0.00      0.00      0.00        14
           5       1.00      0.01      0.03    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


MTMS,▁
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
task1_acc,▁▂▃▃▄▄▅▅▆▆▆▇▇▇██████
task1_accuracy,▁
task1_macro_f1,▁
task1_weighted_f1,▁
task2_acc,▁▂▂▃▃▄▄▅▅▆▆▆▆▇▇▇▇███
task2_accuracy,▁
task2_macro_f1,▁
task2_weighted_f1,▁
task3_acc,▁▄▅▆▇███████████████



Training and evaluation complete!
